In [1]:
import pandas as pd
import re
import gensim.downloader as model_api
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train = pd.read_csv('Trains.csv', skiprows = 1)
train

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [2]:
replaceDict = dict({
'{':" ", '}':" ", ',':"", '.':" ", '!':" ", '\\':" ", '/':" ", '$':" ", '%':" ",
'^':" ", '?':" ", '\'':" ", '"':" ", '(':" ", ')':" ", '*':" ", '+':" ", '-':" ",
'=':" ", ':':" ", ';':" ", ']':" ", '[':" ", '`':" ", '~':" ",
})

rep = dict((re.escape(k), v) for k, v in replaceDict.items())

pattern = re.compile("|".join(rep.keys()))
def replacer(text):
    return rep[re.escape(text.group(0))]

words = train.review.str.replace(pattern, replacer).str.lower().str.split()
words = pd.DataFrame(words.tolist())
words

,0,1,2,3,4,5,6,7,8,9,...,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518
0,with,all,this,stuff,going,down,at,the,moment,with,...,None,None,None,None,None,None,None,None,None,None
1,the,classic,war,of,the,worlds,by,timothy,hines,is,...,None,None,None,None,None,None,None,None,None,None
2,the,film,starts,with,a,manager,nicholas,bell,giving,welcome,...,None,None,None,None,None,None,None,None,None,None
3,it,must,be,assumed,that,those,who,praised,this,film,...,None,None,None,None,None,None,None,None,None,None
4,superbly,trashy,and,wondrously,unpretentious,80,s,exploitation,hooray,the,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,it,seems,like,more,consideration,has,gone,into,the,imdb,...,None,None,None,None,None,None,None,None,None,None
24996,i,don,t,believe,they,made,this,film,completely,unnecessary,...,None,None,None,None,None,None,None,None,None,None
24997,guy,is,a,loser,can,t,get,girls,needs,to,...,None,None,None,None,None,None,None,None,None,None
24998,this,30,minute,documentary,buñuel,made,in,the,early,1930,...,None,None,None,None,None,None,None,None,None,None


In [3]:
#pip install --upgrade gensim

In [4]:
word_vectors = model_api.load("word2vec-google-news-300")

words.columns = words.columns.astype(str)

def soft_get(w):
    try:
        return word_vectors[w]
    except KeyError:
        return np.zeros(word_vectors.vector_size)
    
def map_vectors(row):
    try:
        return np.sum(
            row.loc[row.notna()].apply(soft_get)
        )
    except:
        return np.zeros(word_vectors.vector_size)
    
emb = pd.DataFrame(words.apply(map_vectors, axis=1))
emb.columns = ['C']
emb = pd.DataFrame(np.array(emb.C.apply(pd.Series)))
emb

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,17.283424,11.937847,18.506580,45.120676,-24.435819,3.551744,12.055645,-27.254570,30.983692,19.554092,...,-24.531746,23.092484,-33.328987,9.722412,-26.716827,-10.303997,2.579872,-19.129990,11.317310,-4.080734
1,4.778992,6.744947,3.544289,15.174940,-8.201431,0.314445,4.249725,-9.231554,7.977154,8.119308,...,-12.727188,3.146690,-12.804749,3.808205,-6.146217,-2.835491,-0.495293,-4.693481,4.241936,-0.824364
2,7.668854,18.400442,2.117029,24.500671,-17.646823,-1.491764,3.763025,-24.287819,18.558378,24.320732,...,-23.039650,6.284547,-23.298676,15.287937,-12.791531,-5.257778,2.578659,-12.364639,8.910461,4.143981
3,13.438416,9.676690,3.184448,28.120438,-22.092400,3.097771,14.640491,-28.468567,23.722717,22.710403,...,-25.556366,4.002686,-23.584404,9.610126,-10.111442,-2.726578,3.336586,-10.713562,16.601911,3.251701
4,17.447418,9.209012,8.393372,27.671641,-22.164581,1.606564,11.384158,-23.531187,22.702066,23.904671,...,-24.457840,10.056759,-29.204399,6.255138,-14.498840,-0.895094,6.454372,-19.625992,0.599186,1.965607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,4.843140,3.083902,0.044777,8.552612,-7.245542,1.479004,4.257339,-4.647678,4.680054,2.775642,...,-5.378468,6.225250,-7.275024,3.712402,-4.262329,-1.908768,4.530167,-3.873884,-0.512589,0.154842
24996,5.863220,2.340229,2.210129,12.314983,-7.772020,1.114616,10.503281,-13.057060,13.101852,12.770050,...,-15.988598,6.903183,-15.212250,2.484120,-5.440849,-1.743176,2.057991,-6.311344,6.623047,-2.365334
24997,2.160698,3.789089,-0.081245,13.001617,-8.620522,-1.689472,2.976837,-6.946411,7.065413,7.312721,...,-7.426193,9.140686,-10.666756,4.078735,-6.753197,0.665543,-0.919044,-7.831268,3.209351,-2.725433
24998,9.051056,3.218176,9.683090,16.531189,-7.922881,3.315762,9.002274,-17.094444,12.638168,7.161171,...,-16.743801,0.952118,-15.382585,2.369106,-11.096809,-1.904205,-0.214203,-5.103699,12.709579,-2.317215


In [5]:
emb = emb.fillna(0)

In [6]:
X = Normalizer().fit_transform(emb)
y = train['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)

In [7]:
clf = RandomForestClassifier(n_estimators = 300)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8192

In [8]:
pipe = Pipeline([
    ('standardscaler', StandardScaler()),
    ('pca', PCA()),
    ('reg', LogisticRegression()),
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA()),
                ('reg', LogisticRegression())])

In [9]:
pipe.score(X_test, y_test)

0.8536

In [16]:
param_grid = [
    {
    'pca__n_components': np.arange(50, 60, 5),
    'reg__fit_intercept': [True, False],
    'reg__C': [2, 5, 3]
}]

grid = GridSearchCV(pipe, param_grid, cv = 4, verbose = 1)
grid.fit(X_train, y_train)
grid.best_params_

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   12.3s finished


{'pca__n_components': 55, 'reg__C': 2, 'reg__fit_intercept': True}

In [17]:
model = grid.best_estimator_

In [11]:
y_pred = model.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

0.844

In [13]:
grid.best_params_

{'pca__n_components': 55, 'reg__C': 5, 'reg__fit_intercept': True}

In [14]:
clf = MLPClassifier(solver = 'lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(50, 50), random_state=1)

In [15]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8532